# Playlist Popularity

# question 7

Is the playlist named “Brazilian Music” popular in Brazil?

Which country loves the most “Brazilian Music” playlist based on customer purchases? 

To answer this, we select all the countries that have purchased the tracks in the playlist named “Brazilian Music”

In [ ]:
sql_query = """
SELECT
  Customer.Country,
  COUNT(Customer.Country) AS BrazilianMusicCount,
  Playlist.Name

FROM InvoiceLine
JOIN Track
  ON InvoiceLine.TrackId = Track.TrackId

JOIN PlaylistTrack
  ON PlaylistTrack.TrackId = Track.TrackId

JOIN Playlist
  ON Playlist.PlaylistId = PlaylistTrack.PlaylistId

JOIN Invoice
  ON Invoice.InvoiceId = InvoiceLine.InvoiceId
  
JOIN Customer
  ON Invoice.CustomerId = Customer.CustomerId

WHERE Playlist.Name LIKE 'Brazilian Music'
GROUP BY 1
ORDER BY 2 DESC
"""

pd.read_sql(sql_query, conn) 

In [ ]:
import seaborn as sns
df_BrazilMusic = pd.read_sql(sql_query, conn)
sns.barplot(x = "Country", y="BrazilianMusicCount",data=df_BrazilMusic);

This graph shows that the playlist named “Brazilian Music” is most liked by USA. But it is also popular in Brazil (rank 2). 

# Album Popularity

# question 8

Find the most popular album: which album includes most hit tracks?

To answer this question, we first retrieved the track name, album name, albumID and trackID for all the albums in the database.

In [ ]:
sql_query = """
SELECT Track.Name,
       Album.Title AS Album,
       Album.AlbumID,
       Track.TrackId
FROM Track INNER JOIN Album
ON Track.AlbumId = Album.AlbumId; 
"""

pd.read_sql(sql_query, conn) 

Then, we listed the track name, the times this track is purchased, the album name it belongs to, albumID and trackID for the top 20 most purchased tracks. 

We find that the album named “A Real Dead One” (albumID: 95) includes four top 20 songs, more than any other albums.

In [ ]:
sql_query = """

    SELECT tr.Name, COUNT(inl.InvoiceLineId), al.Title, al.AlbumId, tr.TrackId
    FROM Track AS tr JOIN Album al
    ON tr.AlbumId = al.AlbumId
    JOIN InvoiceLine AS inl
    WHERE tr.TrackId = Inl.TrackId
    GROUP BY tr.Name
    ORDER BY COUNT(inl.InvoiceLineId) DESC
    Limit 20;
"""

pd.read_sql(sql_query, conn)# what albums include top 20 most purchased tracks?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

y = np.array([25, 75])

plt.pie(y,
        labels=['album 95','other albums'], # labels 
        colors=["#d5695d", "#5d8ca8"], # colors
       )
plt.title("which album includes most hit songs?") # pie chart title
plt.show()

# question 9

We can see from the dataset that customers make their purchases by purchasing individual tracks/songs instead of the whole album. This means that chinook customers could purchase any single track they like, without having to purchase the whole album for just one/two tracks they really want (this could be wonderful! You don't need to pay for other tracks you are less interested in in the album).

Let us now have a look at how customers make their purchases: whether they tend to buy individual tracks or not?

In [35]:
sql_query = '''
WITH invoice_first_track AS
    (
     SELECT
         il.invoiceid invoiceid,
         MIN(il.trackid) first_trackid
     FROM invoiceline il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoiceid) number_of_invoices,
    (CAST(count(invoiceid) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice) * 100
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.trackid FROM track t
                  WHERE t.albumid = (
                                      SELECT t2.albumid FROM track t2
                                      WHERE t2.trackid = ifs.first_trackid
                                     ) 

                  EXCEPT 

                  SELECT il2.trackid FROM invoiceline il2
                  WHERE il2.invoiceid = ifs.invoiceid
                 ) IS NULL
             AND
                 (
                  SELECT il2.trackid FROM invoiceline il2
                  WHERE il2.invoiceid = ifs.invoiceid

                  EXCEPT 

                  SELECT t.trackid FROM track t
                  WHERE t.albumid = (
                                      SELECT t2.albumid FROM track t2
                                      WHERE t2.trackid = ifs.first_trackid
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
album_purchase;
'''
pd.read_sql(sql_query, conn)

,album_purchase,number_of_invoices,percent
0,no,412,100.0


Individual track purchases account for 100% of purchases!

# Artist Popularity

# question 10

Who are the top 20 best selling artist in the five years (2009-2013) and how much do these artists earn from their music over these years?

To answer this question, we select the top 20 best selling artists according to the sales amount of their tracks, and we list the income each of these artists(composer/band) earn from their tracks. 

In [ ]:
sql_query = """
SELECT
  ar.Name AS ComposerName_BandName,
  SUM(tr.UnitPrice) AS Income
FROM Track tr
JOIN InvoiceLine in_line
  ON tr.TrackId = in_line.TrackId
JOIN Album al
  ON al.AlbumId = tr.AlbumId
JOIN Artist ar
  ON ar.ArtistId = al.ArtistId
GROUP BY 1
ORDER BY 2 DESC
LIMIT 20;
"""

pd.read_sql(sql_query, conn)

In [ ]:
import seaborn as sns
df = pd.read_sql(sql_query, conn)
sns.barplot(x = "ComposerName_BandName", y="Income",data=df);

This bar chart visualizes our query result, and it shows that “Iron Maiden” is the best selling artist over these years! 

Also, since the top 4 best selling artists are all rock band, this again exemplifies that rock music is really popular! 